In [29]:
import os
import sys
import pandas as pd
import numpy as np
import re

In [2]:
import matplotlib.pyplot as plt

In [3]:
import cv2

In [4]:
#from mtcnn import MTCNN
#mtcnn_detector = MTCNN()

In [5]:
import dlib 
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [6]:
# import detectron2 libraries
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.utils.visualizer import ColorMode
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

In [7]:
subject_folders = os.listdir("Bitirme Projesi Deneyler/")

## Create New Directory

In [8]:
extracted_images_path = "Subject_Images"
if not os.path.exists(extracted_images_path):
    os.makedirs(extracted_images_path)
else:
    print("Directory already created.")

Directory already created.


In [9]:
def rotate_and_scale_frame(img, scaleFactor = 0.5, degreesCCW = 30):
    (oldY,oldX,oldd) = img.shape #note: numpy uses (y,x) convention but most OpenCV functions use (x,y)
    M = cv2.getRotationMatrix2D(center=(oldX/2,oldY/2), angle=degreesCCW, scale=scaleFactor) #rotate about center of image.

    #choose a new image size.
    newX,newY = oldX*scaleFactor,oldY*scaleFactor
    #include this if you want to prevent corners being cut off
    r = np.deg2rad(degreesCCW)
    newX,newY = (abs(np.sin(r)*newY) + abs(np.cos(r)*newX),abs(np.sin(r)*newX) + abs(np.cos(r)*newY))

    #the warpAffine function call, below, basically works like this:
    # 1. apply the M transformation on each pixel of the original image
    # 2. save everything that falls within the upper-left "dsize" portion of the resulting image.

    #So I will find the translation that moves the result to the center of that region.
    (tx,ty) = ((newX-oldX)/2,(newY-oldY)/2)
    M[0,2] += tx #third column of matrix holds translation, which takes effect after rotation.
    M[1,2] += ty

    rotatedImg = cv2.warpAffine(img, M, dsize=(int(newX),int(newY)))
    return rotatedImg

In [10]:
def save_frame(video_path, img_folder_path):
    # Capture video
    cap = cv2.VideoCapture(video_path)
    # get first frame
    pos_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)
    
    # Capture frames and save it.
    while True:
        # get frame
        flag, frame = cap.read()
        
        if flag:
            # get frame number
            pos_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)
            # rotate frame
            frame = cv2.flip(frame, flipCode=cv2.ROTATE_90_COUNTERCLOCKWISE)
            # save frame
            cv2.imwrite(img_folder_path + "/frame%d.jpg" % pos_frame, frame)
        
        else:
            # The next frame is not ready, so we try to read it again
            cap.set(cv2.CAP_PROP_POS_FRAMES, pos_frame-1)
            # It is better to wait for a while for the next frame to be ready
            cv2.waitKey(1000)
        
        if cv2.waitKey(10) == 27:
            break
        
        if cap.get(cv2.CAP_PROP_POS_FRAMES) == cap.get(cv2.CAP_PROP_FRAME_COUNT):
            # If the number of captured frames is equal to the total number of frames,
            # we stop
            break
    # stop capturing video
    cap.release()
    # destroy all windows
    cv2.destroyAllWindows()

In [11]:
def create_extracted_experiment_dir(user_name, experiment_name):
    user_extracted_img_folder = extracted_images_path + "/" + user_name
    extracted_experiment_folder = user_extracted_img_folder + "/" + experiment_name
    if not os.path.exists(user_extracted_img_folder):
        os.makedirs(user_extracted_img_folder)
        os.makedirs(extracted_experiment_folder)
    
    if not os.path.exists(extracted_experiment_folder):
        os.makedirs(extracted_experiment_folder)
    
    return extracted_experiment_folder

In [12]:
def create_extracted_experiment_subdir(user_name, experiment_name, subdir_name):
    # create or retain extracted experiment folder 
    exp_dir = create_extracted_experiment_dir(user_name, experiment_name)
    # create frame directory
    subdir_path = exp_dir + "/" + subdir_name
    if not os.path.exists(subdir_path):
        os.makedirs(subdir_path)
    # return created frame directory
    return subdir_path

In [13]:
def extract_experiment_frame(video_path, user_name, experiment_name):
    # create frame directory
    frame_dir = create_extracted_experiment_subdir(user_name, experiment_name, subdir_name="frame")
    # save video as frames
    save_frame(video_path, frame_dir)
    return frame_dir

In [14]:
def get_margin(image, margin=5):
    y_margin = (image.shape[0] // 100) * margin
    x_margin = (image.shape[1] // 100) * margin
    return x_margin, y_margin

In [15]:
def find_bounds(points, image):
    x_margin, y_margin = get_margin(image)
    x_center, y_center = points[0], points[1]
    
    x0 = x_center - x_margin
    x1 = x_center + x_margin
    
    y0 = y_center - y_margin
    y1 = y_center + y_margin
    
    return x0, x1, y0, y1

In [16]:
def extract_eyes_from_image_mtcnn(image, pos_frame, left_eye_path, right_eye_path):
    # MTCNN output
    result = mtcnn_detector.detect_faces(image)
    
    if result == []:
        return
    
    # get locations about MTCNN output
    keypoints = result[0]['keypoints']
    
    # get left eye image
    x0, x1, y0, y1 = find_bounds(keypoints['left_eye'], image)
    left_eye = image[y0:y1, x0:x1]
    
    # get right eye image
    x0, x1, y0, y1 = find_bounds(keypoints['right_eye'], image)
    right_eye = image[y0:y1, x0:x1]

    # save left eye
    cv2.imwrite(left_eye_path + "/" + "left_eye%d.jpg" % pos_frame, left_eye)
    
    # save right eye
    cv2.imwrite(right_eye_path + "/" + "right_eye%d.jpg" % pos_frame, right_eye)

In [17]:
def dlib_apply_margin(image, x1, x2, y1, y2):
    x_margin, y_margin = get_margin(image, margin=5)
    x1 = x1 - x_margin
    x2 = x2 + x_margin
    
    y1 = y1 - y_margin
    y2 = y2 + y_margin
    
    return image[y1:y2, x1:x2]

In [18]:
def extract_eyes_from_image_dlib(image, pos_frame, left_eye_path, right_eye_path):
    # start detector and find rectangles
    detect = detector(image,1)
    # get facial landmark positions from rectangles
    try:
        shape = predictor(image, detect[0])
    except IndexError:
        print("Error at {0} frame".format(pos_frame))
        return
    
    # extract left eye landmark positions
    x1 = shape.part(37).x
    x2 = shape.part(40).x
    y1 = shape.part(38).y
    y2 = shape.part(42).y
    # construct left eye image with positions
    left_eye = dlib_apply_margin(image, x1, x2, y1, y2)
    
    # extract right eye landmark positions
    x1 = shape.part(43).x
    x2 = shape.part(46).x
    y1 = shape.part(44).y
    y2 = shape.part(45).y
    # construct right eye image with positions
    right_eye = dlib_apply_margin(image, x1, x2, y1, y2)
    
    if 0 in left_eye.shape or 0 in right_eye.shape:
        print("{0} image empty.".format(pos_frame))
        return
    
    # save left eye
    cv2.imwrite(left_eye_path + "/" + "left_eye%d.jpg" % pos_frame, left_eye)
    
    # save right eye
    cv2.imwrite(right_eye_path + "/" + "right_eye%d.jpg" % pos_frame, right_eye)

In [19]:
import time

In [20]:
def extract_experiment_eyes(frame_dir, user_name, experiment_name):
    # create paths for extracted images with mtcnn
    #left_eyes_mtcnn = create_extracted_experiment_subdir(user_name, experiment_name, "left_eye_mtcnn")
    #right_eyes_mtcnn = create_extracted_experiment_subdir(user_name, experiment_name, "right_eye_mtcnn")
    # create paths for extracted images with dlib
    left_eyes_dlib = create_extracted_experiment_subdir(user_name, experiment_name, "left_eye_dlib")
    right_eyes_dlib = create_extracted_experiment_subdir(user_name, experiment_name, "right_eye_dlib")

    for frame_name in os.listdir(frame_dir):
        pos_frame = int (frame_name.replace("frame", "").replace(".jpg", ""))
        image = cv2.imread(frame_dir + "/" + frame_name)
        rotated = rotate_and_scale_frame(image, degreesCCW=-90)
        extract_eyes_from_image_dlib(rotated, pos_frame, left_eyes_dlib, right_eyes_dlib)
        #extract_eyes_from_image_mtcnn(rotated, pos_frame, left_eyes_mtcnn, right_eyes_mtcnn)

In [21]:
#extract_experiment_eyes("Subject_Images/ibrahim/multiply_easy0/frame/", "ibrahim", "multiply_easy0")

In [22]:
def prepare_detectron_model():
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file('COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml'))
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.85# Set threshold for this model
    cfg.MODEL.WEIGHTS = 'output/model_final.pth' # Set path model .pth
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
    detectron_model = DefaultPredictor(cfg)
    
    # register dataset to get label as "pupil"
    try:
        register_coco_instances("all", {}, "images.json", "images/")
    except AssertionError:
        pass
    
    aaatest = MetadataCatalog.get("all")
    dataset_dicts = DatasetCatalog.get("all")
    cfg.DATASETS.TRAIN = ("all",)
    
    return detectron_model, cfg

In [60]:
for subject in subject_folders:
    experiment_folder_path = "Bitirme Projesi Deneyler/"+ subject+"/"
    experiments = sorted(os.listdir(experiment_folder_path))
    for experiment in experiments:
        
        experiment_path = experiment_folder_path + experiment
        print(experiment_path)
        experiment_name = experiment.replace(subject, "").replace(".mp4", "")[1:]
        if os.path.exists("Subject_Images/"+subject+"/"+experiment_name):
            continue
        else:
            start = time.time()
            frame_dir = extract_experiment_frame(experiment_path, subject, experiment_name)
            extract_experiment_eyes(frame_dir, subject, experiment_name)
            end = time.time()
            print(end - start)

Bitirme Projesi Deneyler/ibrahim/ibrahim_fear0.mp4
Bitirme Projesi Deneyler/ibrahim/ibrahim_fear1.mp4
Bitirme Projesi Deneyler/ibrahim/ibrahim_fear2.mp4
Bitirme Projesi Deneyler/ibrahim/ibrahim_happiness0.mp4
Bitirme Projesi Deneyler/ibrahim/ibrahim_happiness1.mp4
Bitirme Projesi Deneyler/ibrahim/ibrahim_happiness2.mp4
Bitirme Projesi Deneyler/ibrahim/ibrahim_light_test0.mp4
Bitirme Projesi Deneyler/ibrahim/ibrahim_multiply_easy0.mp4
Bitirme Projesi Deneyler/ibrahim/ibrahim_multiply_hard0.mp4
88.23754954338074
Bitirme Projesi Deneyler/ibrahim/ibrahim_normal_test0.mp4
90.47955393791199
Bitirme Projesi Deneyler/ibrahim/ibrahim_patriotic0.mp4
96.24590396881104
Bitirme Projesi Deneyler/ibrahim/ibrahim_patriotic1.mp4
97.09651303291321
Bitirme Projesi Deneyler/ibrahim/ibrahim_patriotic2.mp4
84.25467777252197
Bitirme Projesi Deneyler/ibrahim/ibrahim_political_attitude0.mp4
86.52736592292786
Bitirme Projesi Deneyler/ibrahim/ibrahim_political_attitude1.mp4
77.36602783203125
Bitirme Projesi Dene

Error at 64 frame
Error at 215 frame
Error at 88 frame
Error at 200 frame
Error at 109 frame
Error at 186 frame
Error at 217 frame
Error at 213 frame
Error at 119 frame
Error at 24 frame
Error at 98 frame
Error at 135 frame
Error at 66 frame
Error at 147 frame
Error at 189 frame
Error at 27 frame
Error at 111 frame
Error at 165 frame
Error at 225 frame
Error at 38 frame
Error at 99 frame
Error at 26 frame
Error at 128 frame
Error at 89 frame
Error at 49 frame
Error at 125 frame
Error at 216 frame
Error at 37 frame
Error at 104 frame
Error at 157 frame
Error at 85 frame
Error at 228 frame
Error at 191 frame
Error at 190 frame
Error at 115 frame
Error at 94 frame
Error at 234 frame
Error at 160 frame
Error at 281 frame
Error at 46 frame
Error at 113 frame
Error at 198 frame
Error at 110 frame
Error at 67 frame
Error at 121 frame
Error at 221 frame
Error at 187 frame
Error at 120 frame
Error at 80 frame
Error at 172 frame
Error at 210 frame
Error at 140 frame
Error at 340 frame
Error at 8

Error at 101 frame
Error at 54 frame
Error at 152 frame
Error at 183 frame
Error at 369 frame
Error at 134 frame
Error at 275 frame
Error at 278 frame
Error at 180 frame
Error at 112 frame
Error at 117 frame
Error at 44 frame
Error at 171 frame
Error at 106 frame
Error at 349 frame
Error at 274 frame
Error at 161 frame
Error at 280 frame
Error at 182 frame
Error at 148 frame
Error at 51 frame
Error at 69 frame
Error at 43 frame
Error at 367 frame
Error at 185 frame
Error at 302 frame
Error at 62 frame
Error at 194 frame
Error at 298 frame
Error at 297 frame
Error at 208 frame
Error at 76 frame
Error at 371 frame
Error at 64 frame
Error at 215 frame
Error at 88 frame
Error at 109 frame
Error at 352 frame
Error at 123 frame
Error at 186 frame
Error at 217 frame
Error at 213 frame
Error at 119 frame
Error at 218 frame
Error at 222 frame
Error at 292 frame
Error at 98 frame
Error at 366 frame
Error at 135 frame
Error at 268 frame
Error at 267 frame
Error at 241 frame
Error at 271 frame
Err

Error at 309 frame
Error at 255 frame
Error at 321 frame
Error at 301 frame
Error at 312 frame
Error at 329 frame
Error at 282 frame
Error at 305 frame
Error at 313 frame
Error at 273 frame
Error at 286 frame
Error at 341 frame
Error at 254 frame
Error at 300 frame
Error at 308 frame
Error at 335 frame
Error at 277 frame
Error at 310 frame
Error at 285 frame
Error at 348 frame
Error at 295 frame
Error at 323 frame
Error at 315 frame
Error at 296 frame
Error at 294 frame
Error at 299 frame
Error at 270 frame
Error at 328 frame
Error at 272 frame
Error at 327 frame
Error at 342 frame
Error at 336 frame
Error at 275 frame
Error at 278 frame
Error at 332 frame
Error at 288 frame
Error at 349 frame
Error at 274 frame
Error at 280 frame
Error at 339 frame
Error at 302 frame
Error at 316 frame
Error at 326 frame
Error at 298 frame
Error at 297 frame
Error at 345 frame
Error at 292 frame
Error at 269 frame
Error at 343 frame
Error at 268 frame
Error at 267 frame
Error at 271 frame
Error at 338

Error at 286 frame
Error at 233 frame
Error at 341 frame
Error at 136 frame
Error at 168 frame
Error at 254 frame
Error at 181 frame
Error at 158 frame
Error at 201 frame
Error at 97 frame
Error at 300 frame
Error at 133 frame
Error at 178 frame
Error at 256 frame
Error at 308 frame
Error at 92 frame
Error at 335 frame
Error at 195 frame
Error at 170 frame
Error at 124 frame
Error at 277 frame
Error at 203 frame
Error at 310 frame
Error at 285 frame
Error at 179 frame
Error at 131 frame
Error at 96 frame
Error at 239 frame
Error at 350 frame
Error at 237 frame
Error at 173 frame
Error at 295 frame
Error at 323 frame
Error at 315 frame
Error at 253 frame
Error at 252 frame
Error at 199 frame
Error at 296 frame
Error at 294 frame
Error at 103 frame
Error at 299 frame
Error at 163 frame
Error at 226 frame
Error at 270 frame
Error at 184 frame
Error at 328 frame
Error at 272 frame
Error at 327 frame
Error at 164 frame
Error at 247 frame
Error at 202 frame
Error at 351 frame
Error at 342 fr

124.65161323547363
Bitirme Projesi Deneyler/ekrem/ekrem_sum_hard0.mp4
Error at 3 frame
Error at 4 frame
Error at 2 frame
135.47825622558594
Bitirme Projesi Deneyler/yusufErdgn/yusufErdgn_fear0.mp4
121.937828540802
Bitirme Projesi Deneyler/yusufErdgn/yusufErdgn_fear1.mp4
Error at 39 frame
Error at 38 frame
118.98017573356628
Bitirme Projesi Deneyler/yusufErdgn/yusufErdgn_fear2.mp4
121.42066955566406
Bitirme Projesi Deneyler/yusufErdgn/yusufErdgn_happiness0.mp4
125.97098445892334
Bitirme Projesi Deneyler/yusufErdgn/yusufErdgn_happiness1.mp4
123.10416150093079
Bitirme Projesi Deneyler/yusufErdgn/yusufErdgn_happiness2.mp4
119.88425540924072
Bitirme Projesi Deneyler/yusufErdgn/yusufErdgn_light_test0.mp4
63.87681579589844
Bitirme Projesi Deneyler/yusufErdgn/yusufErdgn_multiply_easy0.mp4
118.63363933563232
Bitirme Projesi Deneyler/yusufErdgn/yusufErdgn_multiply_hard0.mp4
128.38714575767517
Bitirme Projesi Deneyler/yusufErdgn/yusufErdgn_normal_test0.mp4
126.61460399627686
Bitirme Projesi Deney

Error at 135 frame
Error at 343 frame
Error at 429 frame
Error at 268 frame
Error at 267 frame
Error at 407 frame
Error at 241 frame
Error at 238 frame
Error at 271 frame
Error at 401 frame
Error at 374 frame
Error at 338 frame
Error at 147 frame
Error at 266 frame
Error at 189 frame
Error at 111 frame
Error at 165 frame
Error at 225 frame
Error at 258 frame
Error at 99 frame
Error at 257 frame
Error at 425 frame
Error at 128 frame
Error at 89 frame
Error at 396 frame
Error at 383 frame
Error at 125 frame
Error at 405 frame
Error at 337 frame
Error at 284 frame
Error at 334 frame
Error at 138 frame
Error at 382 frame
Error at 403 frame
Error at 156 frame
Error at 216 frame
Error at 360 frame
Error at 104 frame
Error at 289 frame
Error at 157 frame
Error at 85 frame
Error at 415 frame
Error at 228 frame
Error at 385 frame
Error at 388 frame
Error at 250 frame
Error at 191 frame
Error at 190 frame
Error at 242 frame
Error at 344 frame
Error at 115 frame
Error at 94 frame
Error at 331 fra

105.57259774208069
Bitirme Projesi Deneyler/Etodk/Etodk_sadness2.mp4
114.99682450294495
Bitirme Projesi Deneyler/Etodk/Etodk_sum_easy0.mp4
101.20552349090576
Bitirme Projesi Deneyler/Etodk/Etodk_sum_hard0.mp4
107.92260026931763


In [112]:
path = "Subject_Images/Ali/fear0/left_eye_dlib/"
start = time.time()
for filename in os.listdir(path):
    image = cv2.imread(path+filename)
    outputs = detect_pupil(detectron_model, cfg, image)
    try:
        current_area = outputs["instances"].pred_boxes[0].area().tolist()[0]
        curent_area = round(current_area, 2)
    except IndexError:
        print("The {0} numbered file couldn't segmentated.".format(filename))
        
end = time.time()

print("Time wasted for 1 eye : {0}".format(end-start))

The left_eye411.jpg numbered file couldn't segmentated.
The left_eye410.jpg numbered file couldn't segmentated.
The left_eye279.jpg numbered file couldn't segmentated.
The left_eye133.jpg numbered file couldn't segmentated.
The left_eye408.jpg numbered file couldn't segmentated.
The left_eye280.jpg numbered file couldn't segmentated.
The left_eye278.jpg numbered file couldn't segmentated.
Time wasted for 1 eye : 125.81317257881165


In [25]:
def create_predicted_img_folder(extracted_img_folder, path):
    
    if "left" in extracted_img_folder:
        if not os.path.exists(path + "/" +"left_eye_predicted"):
            os.makedirs(path + "/"+ "left_eye_predicted")
            return path + "/" + "left_eye_predicted"
        else:
            return path + "/" + "left_eye_predicted"
    
    if "right" in extracted_img_folder:
        if not os.path.exists(path + "/" + "right_eye_predicted"):
            os.makedirs(path + "/" + "right_eye_predicted")
            return path + "/" + "right_eye_predicted"
        else:
            return path + "/" + "right_eye_predicted"
    
    else:
        raise ValueError("Wrong folder name: {0}".format(extracted_img_folder))

In [58]:
def detect_pupil(detectron_model, cfg, img_path, result_path, filename):
    
    image = cv2.imread(img_path + "/" + filename)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    outputs = detectron_model(image)
        
    v = Visualizer(image[:, :, ::-1],
               MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), 
               scale=4, 
               instance_mode=ColorMode.IMAGE)
    
    try:
        current_area = outputs["instances"].pred_boxes[0].area().tolist()[0]
        curent_area = round(current_area, 2)
        out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        predicted_image = out.get_image()[:, :, ::-1]
        
        cv2.imwrite(result_path + "/" + filename, cv2.cvtColor(predicted_image, cv2.COLOR_BGR2RGB))

        return curent_area 
        
    except IndexError:
        pass
        #print("The {0} numbered file couldn't segmentated.".format(filename))
    
    
    return 0

In [77]:
detectron_model, cfg = prepare_detectron_model()
subject_path = "Subject_Images/"
# denekler : Ali, yusuf ...
for subject in sorted(os.listdir(subject_path)):

    # deneklere ait dosyalar: fear0, fear1 ....
    experiment_folder = subject_path + subject
    for experiment in sorted(os.listdir(experiment_folder)):
        
        # light test'i segmente etmeye gerek yok
        if "light_test" in experiment:
            continue

        
        experiment_data = pd.DataFrame(columns=["left_eye_area", "right_eye_area"])
        # fear0 içerisinde olan dosyalar : frame, left_eye_dlib, right_eye_dlib
        extracted_folders = experiment_folder + "/" + experiment
        
        if experiment+"_areas.csv" in (os.listdir(extracted_folders)):
            continue
        
        for image_folder in sorted(os.listdir(extracted_folders)):
            # dosya dlib ismini içeriyorsa, frame, predicted giremez
            if "dlib" in image_folder:
                # predict için dosya oluştur
                result_path = create_predicted_img_folder(image_folder, extracted_folders)
                
                # segmente edilecek resimleri bul
                img_path = extracted_folders + "/" + image_folder
                images = os.listdir(img_path)
                # segmente edilecek resimleri sırala
                images.sort(key=lambda f: int(re.sub('\D', '', f)))
                print(img_path)
                # sayacı başlat
                start = time.time()
                # segmentasyonu başlat
                areas = [detect_pupil(detectron_model, cfg, img_path, result_path, x) for x in images]
                if "left" in image_folder:
                    experiment_data["left_eye_area"] = areas
                
                if "right" in image_folder:
                    experiment_data["right_eye_area"] = areas
                
                
                # sayacı durdur
                end = time.time()
                
                print("Time spent: {0}".format(end-start))
        
        experiment_data.to_csv(extracted_folders + "/" + experiment+"_areas.csv", index=False)
        

Subject_Images/Ali/fear1/left_eye_dlib
Time spent: 213.02883195877075
Subject_Images/Ali/fear1/right_eye_dlib
Time spent: 213.4679298400879
Subject_Images/Ali/fear2/left_eye_dlib
Time spent: 167.15785479545593
Subject_Images/Ali/fear2/right_eye_dlib
Time spent: 147.8765366077423
Subject_Images/Ali/happiness0/left_eye_dlib
Time spent: 139.51075077056885
Subject_Images/Ali/happiness0/right_eye_dlib
Time spent: 145.8989450931549
Subject_Images/Ali/happiness1/left_eye_dlib
Time spent: 145.42885851860046
Subject_Images/Ali/happiness1/right_eye_dlib
Time spent: 153.23033952713013
Subject_Images/Ali/happiness2/left_eye_dlib
Time spent: 118.26590037345886
Subject_Images/Ali/happiness2/right_eye_dlib
Time spent: 119.65417242050171
Subject_Images/Ali/multiply_easy0/left_eye_dlib
Time spent: 125.82251477241516
Subject_Images/Ali/multiply_easy0/right_eye_dlib
Time spent: 126.84980320930481
Subject_Images/Ali/multiply_hard0/left_eye_dlib
Time spent: 133.03271198272705
Subject_Images/Ali/multiply_ha

Time spent: 104.5476222038269
Subject_Images/RebelKing/political_attitude2/left_eye_dlib
Time spent: 87.47990894317627
Subject_Images/RebelKing/political_attitude2/right_eye_dlib
Time spent: 86.20610284805298
Subject_Images/RebelKing/sadness0/left_eye_dlib
Time spent: 96.30664706230164
Subject_Images/RebelKing/sadness0/right_eye_dlib
Time spent: 97.05196595191956
Subject_Images/RebelKing/sadness1/left_eye_dlib
Time spent: 109.09677028656006
Subject_Images/RebelKing/sadness1/right_eye_dlib
Time spent: 108.77388882637024
Subject_Images/RebelKing/sadness2/left_eye_dlib
Time spent: 95.60332655906677
Subject_Images/RebelKing/sadness2/right_eye_dlib
Time spent: 94.80118155479431
Subject_Images/RebelKing/sum_easy0/left_eye_dlib
Time spent: 94.06338667869568
Subject_Images/RebelKing/sum_easy0/right_eye_dlib
Time spent: 97.9722192287445
Subject_Images/RebelKing/sum_hard0/left_eye_dlib
Time spent: 85.67434930801392
Subject_Images/RebelKing/sum_hard0/right_eye_dlib
Time spent: 89.14858055114746
S

Time spent: 114.60808253288269
Subject_Images/fatih/multiply_hard0/left_eye_dlib
Time spent: 108.41632318496704
Subject_Images/fatih/multiply_hard0/right_eye_dlib
Time spent: 112.04645419120789
Subject_Images/fatih/normal_test0/left_eye_dlib
Time spent: 75.80923461914062
Subject_Images/fatih/normal_test0/right_eye_dlib
Time spent: 82.39962935447693
Subject_Images/fatih/patriotic0/left_eye_dlib
Time spent: 126.19532084465027
Subject_Images/fatih/patriotic0/right_eye_dlib
Time spent: 121.67940616607666
Subject_Images/fatih/patriotic1/left_eye_dlib
Time spent: 111.79472041130066
Subject_Images/fatih/patriotic1/right_eye_dlib
Time spent: 110.3380389213562
Subject_Images/fatih/patriotic2/left_eye_dlib
Time spent: 106.425222158432
Subject_Images/fatih/patriotic2/right_eye_dlib
Time spent: 106.63065910339355
Subject_Images/fatih/political_attitude0/left_eye_dlib
Time spent: 106.88332462310791
Subject_Images/fatih/political_attitude0/right_eye_dlib
Time spent: 111.36801338195801
Subject_Images

Time spent: 98.70113635063171
Subject_Images/kral/sum_hard0/right_eye_dlib
Time spent: 106.7629189491272
Subject_Images/yusufErdgn/fear0/left_eye_dlib
Time spent: 109.13505792617798
Subject_Images/yusufErdgn/fear0/right_eye_dlib
Time spent: 103.29663443565369
Subject_Images/yusufErdgn/fear1/left_eye_dlib
Time spent: 103.10563206672668
Subject_Images/yusufErdgn/fear1/right_eye_dlib
Time spent: 100.99793696403503
Subject_Images/yusufErdgn/fear2/left_eye_dlib
Time spent: 99.79707336425781
Subject_Images/yusufErdgn/fear2/right_eye_dlib
Time spent: 97.33423376083374
Subject_Images/yusufErdgn/happiness0/left_eye_dlib
Time spent: 105.53681087493896
Subject_Images/yusufErdgn/happiness0/right_eye_dlib
Time spent: 102.75445747375488
Subject_Images/yusufErdgn/happiness1/left_eye_dlib
Time spent: 104.0763156414032
Subject_Images/yusufErdgn/happiness1/right_eye_dlib
Time spent: 100.54598188400269
Subject_Images/yusufErdgn/happiness2/left_eye_dlib
Time spent: 97.03608393669128
Subject_Images/yusufEr

In [51]:
detectron_model?

In [57]:
len(areas)

417

In [65]:
pd.DataFrame(areas)

,0
0,78.83
1,0.00
2,76.19
3,45.47
4,38.40
...,...
421,26.11
422,43.34
423,75.36
424,53.71


In [75]:
if experiment+"_areas.csv" in os.listdir(experiment_folder + "/" + experiment):
    print("yes")

yes


In [74]:
os.listdir(experiment_folder + "/" + experiment)

['left_eye_dlib',
 'frame',
 'right_eye_predicted',
 'right_eye_dlib',
 'left_eye_predicted',
 'fear0_areas.csv']